# Tensorflow Overview

TF is an open source software library used for high performance Numerical Computation. It originally created by the Google Brain Team as an internal machine learning tool but an implementation of it was open sourced  in Nov 2015 under Apache 2.0 Licence

### What Makes TF So Great?

- Flexibility: easy deployment of computations across multiple platforma ( CPUs, GPUs and TPUs )
- Scalable: It was originally deployed by Google as a single infrastructure in both production and research.
- Popularity: TF has the largest community with the higest number of repository and stars. It can be deployed both on web and mobile.
- Widely adopted by big companies like Intel, AirBnB, Snapchat etc

---------------------------------------------

**We'd be majorly using CS20 course material and syllabus throughout this second AI6Lagos Cohort http://web.stanford.edu/class/cs20si/syllabus.html** <br>
Thank you Chip and cs20 staff :)

---------------------

### Setup

follow the instruction on https://www.tensorflow.org/install/

If you already have TF installed and want to upgrade to the latest version <br>
<br>
```pip install -q --upgrade tensorflow```

---------------------------------------------------------

### Goals

- Understand TF computation graph approach
- Explore some of TF built-in functions and classes
- Learn how to build and structure models best suited for a deep learning project

------------------------------------------

#### Getting Started

In [1]:
import tensorflow as tf

In [ ]:
tf.__version__

------------------------------------

#### Introduction

Tensorflow has two major different APIs
1. High level
2. Low level

High level Apis act as a wrapper for a lot of low level Apis. Low level means you are explicitly creating your own graphs and sessions
<br>
----> We'd be focusing on Low level Apis for now because it'd help us understand how things work internally once we start using high level Apis


---------------------------------------------------------------------------------------

What we'd learn:

- how to manage our own tensorflow program using graphs and sessions
- how to run tensorflow operations using sessions

#### Tensors

A tensor is simply some data. So TensorFlow can be seen as Data-flow i.e data moving from point A to B
<img src="images/data-flow.jpg">
img source: [cs20](https://docs.google.com/presentation/d/1dizKPtp9hkuTwVDzoGZdYQb_61ULSsSUvaFfDFuhIc4/edit#slide=id.g1bcfa4d819_0_81)

In the image above, each **edge** represent tensor and a **node** represent some operations. A node receives tensors and gives out tensors. <br><br>
TF builds something called a computational graph which consists of tensors and nodes. Then you execute the graph in a session. <br>
Unlike some deep learning frameworks, TF seperates definition of computations from its execution which leads to a non-dynamic graphs but with recent version, we will be able to build a dynamic graph in TF using something called <b>eager execution</b>

#### How the Graph-based approach works
- Assemble a graph
- Use session to execute operations in the graph

---------------------------------------------

Since we get the idea of what tensor means, if I have y = 1, we can say my scalar variable y is a tensor right?
<br>
Exactly!

##### Some unique attributes of Tensors

1. When writing a TF program, the main object you manipulate and pass around is a tensor
2. A tensor object is a partially defined computation that eventually produces an output
3. Each element in the tensor object has the same data type and the data type is always known e.g float32, string or int32
4. The shape and the number of dimension a tensor has might be only partially known but most operations produce tensors of fully known shapes if the shapes of their inputs are also fully known but in some cases it is only possible to find the shape of a tensor when you execute the graph.
5. Tensors values cannot be manupulated once defined except tf.Variable

---------------------------------------
##### Special Tensors
1. tf.Variable
2. tf.constant
3. tf.placeholder
4. tf.SparseTensor
--------------------------

#### Rank 0 tensor

In [2]:
mammal = tf.Variable("Elephant", tf.string, name="my_elephant")
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

In [ ]:
# this allows us to run the code interactively. More on sessions below.
# sess = tf.InteractiveSession() 


In [3]:
# this is just some declaration, can't be executed yet
mammal

<tf.Variable 'my_elephant:0' shape=() dtype=string_ref>

In [4]:
mammal.get_shape()

TensorShape([])

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.initialize_all_variables())
#     print (sess.run(mammal))

-----------------

#### Rank 1 tensor

In [ ]:
mystr = tf.Variable(["Hello"], tf.string)
mystr.get_shape()

#### Rank 2 tensor

In [ ]:
mystr2 = tf.Variable([["Hello"],["World"]], tf.string)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
mystr2.get_shape()

#### Higher Rank

In [ ]:
my_image = tf.zeros([10, 299, 299, 3]) 

----------

#### Graphs
Computational graph is a series of TensorFlow operations arranged in a graph. It is composed of two types of Objects
1. Tensor Objects: edges of the graph
2. Operations (or Ops): Nodes of the graphs
------------------------
Let's build a graph: ->
The most basic CG is a constant

In [ ]:
a = tf.constant(3, dtype=tf.int32)
b = tf.constant(5) # automagically figures the dtype out
c = tf.add(a, b)

------------------------------------------------

Let's Visualize this using TensorBoard

In [ ]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [ ]:
# graph on http://localhost:6006/
# ! tensorboard --logdir .

### Sessions
tf.Session() encapsulate the environment in which you can execute the `operation objects` and evaluate the `tensor objects` <br>

Note: **You allocate memory to a session to store current values of variables, so it's important to close a session once its not in use**

In [ ]:
sess = tf.Session()
sess.run(c) # or c.eval()

In [ ]:
# remeber to close the sess
sess.close()

In [ ]:
# or you can use context manager
with tf.Session() as sess:
    print (sess.run(c))

In [ ]:
# To keep a session interactively, we can use tf.InteractiveSession() 

-----------------------------------------------------

#### Sub graphs

In [ ]:
x = 2
y = 3
add_op = tf.add(x, y)
mult_op = tf.multiply(x, y)
pow_op = tf.pow(add_op, mult_op)

with tf.Session() as sess:
    z = sess.run(pow_op)
    print (z)

In [ ]:
writer.add_graph(tf.get_default_graph())

<img src="images/graph_1.png"></img>

Sub graphs makes it easier to break graphs into several chunks across multiple CPUs, GPUs, TPUs and devices. This leads to distributed computations

In [ ]:
with tf.device('/cpu'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], name='a')
    b = tf.constant([3.0, 4.0, 9.0, 8.0, 5.0], name='b')
    c = tf.multiply(a, b)

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(c)

In [ ]:
sess.close()

-------------------------------

#### What if I want to build more than one graph?

According to [Chip](https://docs.google.com/presentation/d/1dizKPtp9hkuTwVDzoGZdYQb_61ULSsSUvaFfDFuhIc4/edit#slide=id.g1bcfa4d819_0_281) answer <br>
You can with tf.Graph() but you don't really need more than one graph. You can build a graph that has many subgraphs.  <br>

Note: **The tf.Session() runs the default graph.**

----------------------

#### Why I shouldn't build more than one graph?
According to Chip's CS20 material :)
1. Multiple graphs require multiple sessions, each will try to use all available resources by default.
2. You can't pass data between them without passing them through python/numpy which doesn't work in distributed.
3. It is better to have disconnected subgraphs within one graph

In [5]:
# default graph
g1 = tf.get_default_graph()

# user created graph
g2 = tf.Graph()

""" Do not to mix default graphs and 
    user created graphs
"""

# add ops to the default graph
with g1.as_default():
    a = tf.constant(3)
    c = tf.multiply(a, 4)

# add ops to the user created graph
with g2.as_default():
    b = tf.constant(5)
    d = tf.multiply(b, 4)

In [6]:
# evaulate graph 1
with tf.Session() as sess:
    print (sess.run(c))

12


In [8]:
# evaluate graph 2: Error!!!! ops is not an element of this graph.
# with tf.Session() as sess:
#     print(sess.run(d))

In [ ]:
g1

In [ ]:
# how do we run the user  created graph then?
with tf.Session(graph=g2) as sess:
    print(sess.run(d))

In [ ]:
# write our user created graph to tensorboard
# writer.add_graph(g)

# run tensorboard
# ! tensorboard --logdir .

------------------------------------------

#### So Why Graphs?

Ref: CS20 & [TF documentation](https://www.tensorflow.org/programmers_guide/graphs)
1. Saves computation i.e it only runs subgraphs that leads to the value you want to fetch
2. It breaks computations into small differential pieces to facilitate auto-differentiation
3. Allows you to use distributed computation by running sub graphs on multiple devices
4. Many common ML models are taught and visuallized as directed graphs

----------

### Sneak Peak into Next Class

#### feeding value to tf.placeholder()

In [ ]:
# shape is unknown
q = tf.placeholder(tf.float32)
y = tf.add(q, q)

In [ ]:
with tf.Session() as sess:
    print (sess.run(y, feed_dict={q: 5}))

#### Manipulating tf.Variable

In [ ]:
state = tf.Variable(0, name="counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (sess.run(state))
    for _ in range(3):
        sess.run(update)
        print (sess.run(state))

Note: If you're coding in low level TF APIs, you must explicitly initialize the variables <br>

-------------

### Next Class
- Basic operations
- Constants and variables
- Data pipeline
- Fun with TensorBoard
- Inputting Data


Credit: Stanford CS20